In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug 29 19:31:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
%%capture
!apt install git-lfs

In [ ]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "hi", split="train+validation")
common_voice_test = load_dataset("common_voice", "hi", split="test")

Downloading:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/hi/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/hi/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd)


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,सिद्धार्थ मेरा अच्छा दोस्त है और हमेशा रहेगा: आलिया
1,उन्होंने दरवाज़ा खोला.
2,"आप अभी झूठ बोल रहे हो, है ना?"
3,"वह घोड़ा नहीं, वह तो गधा है।"
4,उसकी अंग्रेज़ी मेरी अंग्रेज़ी से अच्छी है।
5,मुझे नहीं पता हमे कितनी देर इंतेज़ार करना चाहिए।
6,अगर हमको स्टेशन समय पर पहुँचना है तो हमको जल्दी करनी होगी।
7,बीजेपी में आज शामिल होंगे भुवनेश्वर कलिता और जी विवेकानंद
8,"अगर वह इस तरह मन लगाकर पढ़ता रहेगा, तो परीक्षा में अवश्य पास होगा।"
9,सर्वदलीय प्रतिनिधिमंडल में शामिल प्रणव व चिदंबरम


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f5512a63950> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,टीवी चालू कीजिए।
1,रसोईघर में।
2,मैं ना एथेनियाई हूँ ना यूनानी।
3,बच्चों को माँबाप की बात माननी चाहिए।
4,प्रदूषण भी है क्राइम बढ़ने की वजह बदल जाता है लोगों का व्यवहार
5,आराम से आओ।
6,उसने तस्वीर को देखा।
7,आदमी सत्तर प्रतिशत पानी का बना हुआ है।
8,सोनम के संगीत से करीना गायब करिश्मावरुण का तारीफां पर डांस
9,वह अमीर है हो है पर कंजूस भी है।


In [ ]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

In [ ]:
common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'e': 2,
 'f': 3,
 'i': 4,
 'l': 5,
 'm': 6,
 'p': 7,
 'r': 8,
 'u': 9,
 'w': 10,
 'ँ': 11,
 'ं': 12,
 'ः': 13,
 'अ': 14,
 'आ': 15,
 'इ': 16,
 'ई': 17,
 'उ': 18,
 'ऊ': 19,
 'ए': 20,
 'ऐ': 21,
 'ऑ': 22,
 'ओ': 23,
 'औ': 24,
 'क': 25,
 'ख': 26,
 'ग': 27,
 'घ': 28,
 'च': 29,
 'छ': 30,
 'ज': 31,
 'झ': 32,
 'ट': 33,
 'ठ': 34,
 'ड': 35,
 'ढ': 36,
 'ण': 37,
 'त': 38,
 'थ': 39,
 'द': 40,
 'ध': 41,
 'न': 42,
 'प': 43,
 'फ': 44,
 'ब': 45,
 'भ': 46,
 'म': 47,
 'य': 48,
 'र': 49,
 'ल': 50,
 'व': 51,
 'श': 52,
 'ष': 53,
 'स': 54,
 'ह': 55,
 '़': 56,
 'ा': 57,
 'ि': 58,
 'ी': 59,
 'ु': 60,
 'ू': 61,
 'ृ': 62,
 'े': 63,
 'ै': 64,
 'ॉ': 65,
 'ो': 66,
 'ौ': 67,
 '्': 68,
 'ड़': 69,
 '।': 70}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

73

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
repo_name="wav2vec2-xlsr-hindi"

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='wav2vec2-xlsr-hindi' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,


HTTPError: ignored

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
common_voice_train[0]["path"]

OSError: ignored

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

OSError: ignored

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

OSError: ignored

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

OSError: ignored